In [1]:

import math
import time
import sys
import os
import random
import pickle

import numpy as np

import tensorflow.keras
import tensorflow.keras.layers
import tensorflow.keras.applications
import tensorflow.keras.backend
import tensorflow.keras.preprocessing.image
import tensorflow.keras.utils
import tensorflow as tf

import cv2
import PIL
import PIL.Image
import matplotlib.pyplot as plt

import shutil

from tensorflow.keras import backend as K


# configurations

## seeding
os.environ['PYTHONHASHSEED'] = '3'
np.random.seed(3)
random.seed(3)
tf.random.set_seed(3)

## which gpu to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## memory allocation
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.compat.v1.Session(config=config)
#K.set_session(session)
tf.compat.v1.keras.backend.set_session(session)
## data directory for CUB200 root
PATH_DATA_ROOT_CUB200 = "/kaggle/working/200-bird-species-with-11788-images/CUB_200_2011"

## network configurations
### number of output classes, 200 for CUB200
NO_CLASS = 200



2022-01-08 16:25:52.725915: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-08 16:25:52.779934: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 16:25:52.869504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 16:25:52.870444: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [2]:
import tarfile
import gzip
import shutil

def extract(tar_file, path):
    opened_tar = tarfile.open(tar_file)
     
    if tarfile.is_tarfile(tar_file):
        opened_tar.extractall(path)
    else:
        print("The tar file you entered is not a tar file")
        
extract('../input/200-bird-species-with-11788-images/CUB_200_2011.tgz', '/kaggle/working/200-bird-species-with-11788-images/')



In [4]:
from tensorflow.keras.initializers import glorot_normal
from keras.layers import Lambda

def outer_product(x):
    """
    calculate outer-products of 2 tensors

        args 
            x
                list of 2 tensors
                , assuming each of which has shape = (size_minibatch, total_pixels, size_filter)
    """
    return tensorflow.keras.backend.batch_dot(
                x[0]
                , x[1]
                , axes=[1,1]
            ) / x[0].get_shape().as_list()[1] 

def signed_sqrt(x):
    """
    calculate element-wise signed square root

        args
            x
                a tensor
    """
    return tensorflow.keras.backend.sign(x) * tensorflow.keras.backend.sqrt(tensorflow.keras.backend.abs(x) + 1e-9)

def L2_norm(x, axis=-1):
    """
    calculate L2-norm

        args 
            x
                a tensor
    """
    return tensorflow.keras.backend.l2_normalize(x, axis=axis)


def build_model(arryWeights_last
    ,size_heigth=448
    ,size_width=448
    ,no_class=200
    ,no_last_layer_backbone=17
    
    ,name_optimizer="sgd"
    ,rate_learning=1.0
    ,rate_decay_learning=0.0
    ,rate_decay_weight=0.0
    
    ,name_initializer="glorot_normal"
    ,name_activation_logits="softmax"
    ,name_loss="categorical_crossentropy"
    ,flg_debug=False
    ,**kwargs
):
    
    tensorflow.keras.backend.clear_session()
    
    # print("-------------------------------")
    # print("parameters:")
    # for key, val in locals().items():
    #     if not val == None and not key == "kwargs":
    #         print("\t", key, "=",  val)
    # print("-------------------------------")
    
    ### 
    ### load pre-trained model
    ###
    tensor_input = tensorflow.keras.layers.Input(shape=[size_heigth,size_width,3])
    model_detector = tensorflow.keras.applications.vgg16.VGG16(
                            input_tensor=tensor_input
                            , include_top=False
                            , weights='imagenet'
                        )
    
    # arryWeights_last_After =[]
    # for i in range(512):
    #   arryWeights_last_After.append(i)
    # for i in range(512):
    #   arryWeights_last_After[i]=1
    
    # arryWeights_last_After[filter_zero]=value_filter
    # print('filter' , filter_zero, "=", arryWeights_last_After[filter_zero])
    def custom_layer_last(tensor):
        return tensor * arryWeights_last
    lambda_layer = Lambda (custom_layer_last, name="lambda_New")

    model_detector = insert_intermediate_layer_in_keras (model_detector, 18, lambda_layer)
    # print('model detector')
    # model_detector.summary()
    ### 
    ### bi-linear pooling
    ###

    # extract features from detector
    x_detector = model_detector.layers[no_last_layer_backbone].output
    shape_detector = model_detector.layers[no_last_layer_backbone].output_shape
    # if flg_debug:
    #     print("shape_detector : {}".format(shape_detector))

    # extract features from extractor , same with detector for symmetry DxD model
    shape_extractor = shape_detector
    x_extractor = x_detector
    # if flg_debug:
    #     print("shape_extractor : {}".format(shape_extractor))
        
    
    # rehape to (minibatch_size, total_pixels, filter_size)
    x_detector = tensorflow.keras.layers.Reshape(
            [
                shape_detector[1] * shape_detector[2] , shape_detector[-1]
            ]
        )(x_detector)
    # if flg_debug:
    #     print("x_detector shape after rehsape ops : {}".format(x_detector.shape))
        
    x_extractor = tensorflow.keras.layers.Reshape(
            [
                shape_extractor[1] * shape_extractor[2] , shape_extractor[-1]
            ]
        )(x_extractor)
    # if flg_debug:
    #     print("x_extractor shape after rehsape ops : {}".format(x_extractor.shape))
        
        
    # outer products of features, output shape=(minibatch_size, filter_size_detector*filter_size_extractor)
    x = tensorflow.keras.layers.Lambda(outer_product)(
        [x_detector, x_extractor]
    )
    # if flg_debug:
    #     print("x shape after outer products ops : {}".format(x.shape))
        
        
    # rehape to (minibatch_size, filter_size_detector*filter_size_extractor)
    x = tensorflow.keras.layers.Reshape([shape_detector[-1]*shape_extractor[-1]])(x)
    # if flg_debug:
    #     print("x shape after rehsape ops : {}".format(x.shape))
        
        
    # signed square-root 
    x = tensorflow.keras.layers.Lambda(signed_sqrt)(x)
    # if flg_debug:
    #     print("x shape after signed-square-root ops : {}".format(x.shape))
        
    # L2 normalization
    x = tensorflow.keras.layers.Lambda(L2_norm)(x)
    # if flg_debug:
    #     print("x shape after L2-Normalization ops : {}".format(x.shape))



    ### 
    ### attach FC-Layer
    ###

    if name_initializer != None:
            name_initializer = eval(name_initializer+"()")
            
    x = tensorflow.keras.layers.Dense(
            units=no_class
            ,kernel_regularizer=tensorflow.keras.regularizers.l2(rate_decay_weight)
            ,kernel_initializer=name_initializer
        )(x)
    # if flg_debug:
    #     print("x shape after Dense ops : {}".format(x.shape))
    tensor_prediction = tensorflow.keras.layers.Activation(name_activation_logits)(x)
    # if flg_debug:
    #     print("prediction shape : {}".format(tensor_prediction.shape))

        

    ### 
    ### compile model
    ###
    model_bilinear = tensorflow.keras.models.Model(
                        inputs=[tensor_input]
                        , outputs=[tensor_prediction]
                    )
    
    
    # fix pre-trained weights
    for layer in model_detector.layers:
        layer.trainable = False
        
        
    # define optimizers
    opt_adam = tensorflow.keras.optimizers.Adam(
                    lr=rate_learning
                    , decay=rate_decay_learning
                )
    opt_rms = tensorflow.keras.optimizers.RMSprop(
                    lr=rate_learning
                    , decay=rate_decay_learning
                )
    opt_sgd = tensorflow.keras.optimizers.SGD(
                    lr=rate_learning
                    , decay=rate_decay_learning
                    , momentum=0.9
                    , nesterov=False
                )
    optimizers ={
        "adam":opt_adam
        ,"rmsprop":opt_rms
        ,"sgd":opt_sgd
    }
    
    model_bilinear.compile(
        loss=name_loss
        , optimizer=optimizers[name_optimizer]
        , metrics=["categorical_accuracy"]
    )
    
    
    
    # if flg_debug:
    #     model_bilinear.summary()
    
    return model_bilinear

In [5]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    model = Model(inputs=layers[0].input, outputs=x) ### inputs instead of input in this version
    return model

In [6]:
arryWeights_last_After =[]
for i in range(512):
  arryWeights_last_After.append(i)
for i in range(512):
  arryWeights_last_After[i]=1
model = build_model(arryWeights_last_After
            # number of output classes, 200 for CUB200
            ,no_class = NO_CLASS

            # pretrained model specification, using VGG16
            # "block5_conv3 "
            ,no_last_layer_backbone = 18

            # training parametes
            ,rate_learning=1.0
            ,rate_decay_weight=1e-8
            ,flg_debug=True
        )

# model = build_model(
#             # number of output classes, 200 for CUB200
#             no_class = NO_CLASS

#             # pretrained model specification, using VGG16
#             # "block5_conv3 "
#             ,no_last_layer_backbone = 17           
#             # training parametes
#             ,rate_learning=1.0
#             ,rate_decay_weight=1e-8
#             ,flg_debug=True
#         )

2022-01-08 16:26:45.113251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 16:26:45.114070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 16:26:45.114748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 16:26:45.115639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 16:26:45.116312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

58900480/58889256 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:
# now all layers are trainable
for layer in model.layers:
    layer.trainable = True

# change LR
opt_sgd = tensorflow.keras.optimizers.SGD(
                lr=1e-3
                , decay=1e-9
                , momentum=0.9
                , nesterov=False
            )
model.compile(
    loss="categorical_crossentropy"
    , optimizer=opt_sgd
    , metrics=["categorical_accuracy"]
)

In [8]:
model.load_weights("../input/birds-weights/birds_200_weights.h5")

In [9]:
import os

birds_labels = set()

path= "./200-bird-species-with-11788-images/CUB_200_2011/images/"
for d in os.listdir(path):
    birds_labels.add(d)

len(birds_labels)

# print(birds_labels)


200

In [10]:
birds_labels = list(birds_labels)
birds_labels.sort()
# birds_labels

In [11]:
import cv2
def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (448,448))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [12]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

def output(location, new_model):
    img = load_and_preprocess_image(location)
#     img = load_img(location, target_size = (224, 224, 3))
#     img = img_to_array(img)
#     img = img / 255
    img = np.expand_dims(img, axis=0)
    answer = new_model.predict(img)
    y_class = answer.argmax(axis = -1)
    
    top_3 = np.argsort(answer[0])[:-4:-1]
    # for i in range(3):
    #     print(" ({:.3})".format(answer[0][top_3[i]]))
    
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = birds_labels[y]
    # print(res)
#     print(" ({:.3})".format(answer[0][top_3[0]]))
    pred_prob = answer[0][top_3[0]]
    return float(pred_prob), res

In [ ]:
location = "./200-bird-species-with-11788-images/CUB_200_2011/images/004.Groove_billed_Ani/Groove_Billed_Ani_0087_1765.jpg"
output(location, model)

In [14]:
import ast
a_file = open("../input/resp-birds/Filt_Resp_Birds_BCNN.txt", "r")
# a_file = open("Gibbon_panda.txt", "r")
# a_file = open("Gimagenet_Test_2.txt", "r")


string_without_line_breaks = ""
for line in a_file:
    stripped_line = line.rstrip()
    string_without_line_breaks += stripped_line

import re
dict_Classes = ast.literal_eval(re.search('({.+})', string_without_line_breaks).group(0))

# for key, value in dict_Classes.items():
#     print ('classId', key)
#     print ('value', value)
a_file.close()
#print(dict_Classes)

In [15]:
import collections
od = collections.OrderedDict(sorted(dict_Classes.items()))
list_b=[]
for item in od:
    # print(item)
    list_b.append(item)

Original **model**

In [ ]:
# import cv2
# import os
# from shutil import copyfile
# from pathlib import Path
# final_acc = 0
# folder = "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/CUB_200_2011/CUB_200_2011/images/"
# print(len(os.listdir(folder + "001.Black_footed_Albatross")))
# imagesFileNames = []
# for filename in os.listdir(folder):    
#     correct_count_By_folder=0
#     print( len(os.listdir(folder+str(filename)) ))
#     for img in os.listdir(folder+filename):
#         if output(folder+filename+"/"+img, model)[1] ==  filename:
#             correct_count_By_folder+=1
#     final_accu_by_class = correct_count_By_folder/len(os.listdir(folder+filename))
#     print('correctly classified : ' ,filename, correct_count_By_folder)
#     print('accuracy for ' , filename , 'is ' , final_accu_by_class)
#     final_acc += final_accu_by_class
#     print('final_acc : ', final_acc/200) &

# #images = load_images_from_folder(folder)


# ** CauSim**

In [16]:
import cv2
import os
from shutil import copyfile
from pathlib import Path
final_acc = 0
folder = "./200-bird-species-with-11788-images/CUB_200_2011/images/"
print(len(os.listdir(folder + "001.Black_footed_Albatross")))
imagesFileNames = []
for filename in os.listdir(folder):
    for key, value in dict_Classes.items():      
        if key == filename: 
          print('key found:', key)
          weights_dic_Conv2_13= {}
          weights_dic_Conv2_13.update(value)
          weights_dic_Conv2_13 = sorted (weights_dic_Conv2_13.items ())
          arryWeights_last_After = []
          od = collections.OrderedDict(weights_dic_Conv2_13)
          for i in range(len(od)):
              if weights_dic_Conv2_13[i][1][0] >0:
                  arryWeights_last_After.append(1) 
              else:
                  arryWeights_last_After.append(0)

          new_model = build_model( arryWeights_last_After
                  # number of output classes, 200 for CUB200
                  ,no_class = NO_CLASS

                  # pretrained model specification, using VGG16
                  # "block5_conv3 "
                  ,no_last_layer_backbone = 18
                              # training parametes
                  ,rate_learning=1.0
                  ,rate_decay_weight=1e-8
                  ,flg_debug=True
              )

                      # now all layers are trainable
          for layer in new_model.layers:
                layer.trainable = True

          # change LR
          opt_sgd = tensorflow.keras.optimizers.SGD(
                          lr=1e-3
                          , decay=1e-9
                          , momentum=0.9
                          , nesterov=False
                      )
          new_model.compile(
              loss="categorical_crossentropy"
              , optimizer=opt_sgd
              , metrics=["categorical_accuracy"]
          )

          new_model.load_weights('../input/birds-weights/birds_200_weights.h5')

          K.clear_session ()
          break;
    correct_count_By_folder=0
    print( len(os.listdir(folder+str(filename)) ))
    for img in os.listdir(folder+filename):
        if output(folder+filename+"/"+img, new_model)[1] ==  filename:
            correct_count_By_folder+=1
    final_accu_by_class = correct_count_By_folder/len(os.listdir(folder+filename))
    print('correctly classified : ' ,filename, correct_count_By_folder)
    print('accuracy for ' , filename , 'is ' , final_accu_by_class)
    final_acc += final_accu_by_class
    print('final_acc : ', final_acc/200) 

#images = load_images_from_folder(folder)


60
key found: 028.Brown_Creeper
59
correctly classified :  028.Brown_Creeper 59
accuracy for  028.Brown_Creeper is  1.0
final_acc :  0.005
key found: 164.Cerulean_Warbler
60
correctly classified :  164.Cerulean_Warbler 60
accuracy for  164.Cerulean_Warbler is  1.0
final_acc :  0.01
key found: 177.Prothonotary_Warbler
60
correctly classified :  177.Prothonotary_Warbler 60
accuracy for  177.Prothonotary_Warbler is  1.0
final_acc :  0.015
key found: 109.American_Redstart
60
correctly classified :  109.American_Redstart 54
accuracy for  109.American_Redstart is  0.9
final_acc :  0.0195
key found: 068.Ruby_throated_Hummingbird
60
correctly classified :  068.Ruby_throated_Hummingbird 57
accuracy for  068.Ruby_throated_Hummingbird is  0.95
final_acc :  0.024249999999999997
key found: 064.Ring_billed_Gull
60
correctly classified :  064.Ring_billed_Gull 54
accuracy for  064.Ring_billed_Gull is  0.9
final_acc :  0.02875
key found: 119.Field_Sparrow
59
correctly classified :  119.Field_Sparrow 55